<a href="https://colab.research.google.com/github/SU-sumico/dlj/blob/main/1_cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

深層学習における基本的実装手順
1. データの読み込み
2. 訓練データと評価データの準備
3. アルゴリズムの選択
4. 学習
5. モデルの評価

目的
* KerasでCNNを構築しMNISTの画像分類を行う。

プログラムで使用するライブラリ/モジュールのインポート

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

データの準備
* MNIST のピクセルの情報は、0〜255 の数値で提供されており、0 が白、255に近くにつれて黒になるような数字になっています。 数値計算の都合上、入力を 0〜1の範囲の数値にした方が良いため、データ型を float に変換したのち、255 で割ります。

正解データの加工
* MNIST の正解データは、対となる画像に何の数字が書かれているかを表す「0」〜「9」の数字を提供してくれています。 しかし、ここで作成する人工知能は前述の通り出力が10個の確率になりますので、今のままでは比較が難しいです。
* そこで「0」〜「9」までの数字の形を変換します。 どのように変換するのかといえば、例えば「4」という数字であれば [0,0,0,0,1,0,0,0,0,0] というような、「4」を表す値だけが「1」となるような1次元配列にします。
* keras.utils.to_categorical: ラベルデータの整数値を2値クラスの行列に変換（例「整数 1」を「0,1,0,0,0,0,0,0,0,0」と表現
* keras.utils.to_categorical: https://keras.io/ja/utils/

In [2]:
# データパラメータ
num_classes = 10
input_shape = (28, 28, 1)

# 1. データの読み込み
# 2. 訓練データと評価データの準備: 訓練セットとテストセットに分ける
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 画像のピクセルの値を 0~1 の間に正規化
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# 画像の形状を確認する（28, 28, 1）
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# クラスベクトルを2値クラス行列に変換する
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


モデルの構築 Sequentialモデル: https://www.tensorflow.org/guide/keras/sequential_model?hl=ja
* 画像分類の問題では、Sequential モデルを使って、画像を入力とし、それがどのクラスに属するかを出力することができます。

In [3]:
# 3. アルゴリズムの選択
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

モデルのコンパイル: https://keras.io/ja/getting-started/sequential-model-guide/
* モデルの学習を始める前に、**compileメソッドを用いどのような学習処理を行なうかを設定する必要があります**。compileメソッドは**3つの引数**を取ります。生成したmodelインスタンスから**compileメソッド**を呼びだします。
 * 最適化アルゴリズム: 引数として、定義されている最適化手法の識別子を文字列として与える（rmspropやadagradなど）、もしくは Optimizerクラスのインスタンスを与えることができます。
 * 損失関数: モデルが最小化しようとする目的関数です。引数として、定義されている損失関数の識別子を文字列として与える（categorical_crossentropyやmseなど）、もしくは目的関数を関数として与えることができます。
 * 評価関数のリスト: 分類問題では精度としてmetrics=['accuracy']を指定したくなるでしょう。引数として、定義されている評価関数の識別子を文字列として与える、もしくは自分で定義した関数を関数として与えることができます。

学習の実行
* 学習するには、生成したmodelインスタンスから**fit()メソッド**を呼びだします。
* model.fit(入力データ、正解ラベル、バッチサイズ、エポック数）
* バッチサイズを小さくすると消費メモリが少なくて済むが、うまく動かなくなる。
* エポックは学習の繰り返す数

* メソッド, compile, fit, evaluate: https://keras.io/ja/models/model/

In [4]:
# 4. 学習
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 16s 6ms/step - loss: 0.3451 - accuracy: 0.8941 - val_loss: 0.0784 - val_accuracy: 0.9795
Epoch 2/15
422/422 [==============================] - 2s 5ms/step - loss: 0.1106 - accuracy: 0.9666 - val_loss: 0.0629 - val_accuracy: 0.9812
Epoch 3/15
422/422 [==============================] - 3s 6ms/step - loss: 0.0860 - accuracy: 0.9731 - val_loss: 0.0472 - val_accuracy: 0.9870
Epoch 4/15
422/422 [==============================] - 2s 6ms/step - loss: 0.0718 - accuracy: 0.9781 - val_loss: 0.0461 - val_accuracy: 0.9877
Epoch 5/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0632 - accuracy: 0.9799 - val_loss: 0.0396 - val_accuracy: 0.9898
Epoch 6/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0568 - accuracy: 0.9819 - val_loss: 0.0370 - val_accuracy: 0.9898
Epoch 7/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0520 - accuracy: 0.9834 - val_loss: 0.0348 - val_accuracy: 0.9902
Epoch

モデルを評価するには、生成したmodelインスタンスから**evaluateメソッド**を呼びだします。

In [5]:
# 5. モデルの評価
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.026976974681019783
Test accuracy: 0.991100013256073
